## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

### select the data

In [4]:
df_lookup = spark.read.format("com.databricks.spark.avro").load("/user/rucio01/dumps/2018-04-30/contents")
df_april = spark.read.json("/user/sprenner/training_official_april.json")

In [5]:
df_lookup.limit(2).toPandas()

SCOPE  \
0  user.apmorris   
1  user.apmorris   

                                                                                      NAME  \
0  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log             
1  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.180791832   

     CHILD_SCOPE  \
0  user.apmorris   
1  user.apmorris   

                                                                                              CHILD_NAME  \
0  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.180791832                 
1  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.13518847.000001.log.tgz   

  DID_TYPE CHILD_TYPE LENGTH   BYTES   ADLER32   MD5 RULE_EVALUATION  \
0  C        D          None   None    None      None  None             
1  D        F          None   194713  7a3ce1cd  None  None             

      UPDATED_AT     CREATED_AT  \
0  1521503596000  1521503574000   
1  1521504986000  1521504960000   

                                                                        GUID  \
0  None                                                                        
1  [197, 197, 33, 131, 178, 97, 76, 109, 133, 13, 90, 49, 147, 122, 96, 121]   

  EVENTS  
0  None   
1  None

In [6]:
df_april.limit(2).toPandas()

account  accountHash clientState  clientStateHash  \
0  jsabater  1852428352   DONE       -878527175         
1  jsabater  1852428352   DONE       -878527175         

                                                                                               dataset  \
0  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709   
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709   

   datasetHash datasetScope  datasetScopeHash          dataset_version  \
0  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709   
1  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709   

   dataset_versionHash datatype  datatypeHash eventType  eventTypeHash  \
0  1352491158           AOD      1206945951    download  59378080        
1  1352491158           AOD      1206945951    download  59378080        

  eventVersion  eventVersionHash                          filename  \
0  1.14.11     -2044360925        AOD.07978428._000112.pool.root.1   
1  1.14.11     -2044360925        AOD.07978428._000310.pool.root.1   

   filenameHash  filesize_double  hits               hostname  hostnameHash  \
0 -59027447      121164705.0      1     nafhh-atlas02.desy.de -322035157      
1  832068753     122792289.0      1     nafhh-atlas02.desy.de -322035157      

  prod_step  prod_stepHash     project  projectHash protocol  protocolHash  \
0  recon     92057515       mc15_14TeV -2057006107   root    -1364943180     
1  recon     92057515       mc15_14TeV -2057006107   root    -1364943180     

          remoteSite  remoteSiteHash run_number  run_numberHash       scope  \
0  FZK-LCG2_DATADISK -1134109429      407099     883092657       mc15_14TeV   
1  FZK-LCG2_DATADISK -1134109429      407099     883092657       mc15_14TeV   

    scopeHash                                      stream_name  \
0 -2057006107  PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100   
1 -2057006107  PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100   

   stream_nameHash                           traceId  traceIdHash  \
0  999304411        a3e2abce18af4da89150f4847d3cbbc1 -460925703     
1  999304411        860787fbb8e64269bac015d36e8b7c2c -1731281430    

                 traceIp  traceIpHash  traceTimeentryUnix  transferStart  \
0  ::ffff:131.169.168.53 -1780265019   1.522841e+09        1.522841e+09    
1  ::ffff:131.169.168.53 -1780265019   1.522842e+09        1.522842e+09    

                               uuid   uuidHash  
0  14a3513ff0ba4dd4be22a7c0476418a8  545236149  
1  14a3513ff0ba4dd4be22a7c0476418a8  545236149

In [7]:
df_april.agg(*(countDistinct(col(c)).alias(c) for c in df_april.columns)).toPandas()

account  accountHash  clientState  clientStateHash  dataset  datasetHash  \
0  539      539          11           11               10578    10578         

   datasetScope  datasetScopeHash  dataset_version  dataset_versionHash  \
0  23            23                3175             3175                  

   datatype  datatypeHash  eventType  eventTypeHash  eventVersion  \
0  112       112           1          1              5              

   eventVersionHash  filename  filenameHash  filesize_double  hits  hostname  \
0  5                 530100    530058        469133           38    344        

   hostnameHash  prod_step  prod_stepHash  project  projectHash  protocol  \
0  344           8          8              22       22           6          

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0  6             164         164             3435        3435             

   scope  scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  \
0  22     22         2954         2954             776984   776908        

   traceIp  traceIpHash  traceTimeentryUnix  transferStart   uuid  uuidHash  
0  327      327          776982              776982         12877  12877

In [8]:
df_april.createOrReplaceTempView("training")
df_sq = spark.sql("SELECT DISTINCT dataset, datasetScope\
                   FROM training")

In [9]:
df_sq.toPandas()

dataset  \
0      data15_13TeV.00284285.physics_Main.merge.DAOD_HIGG4D4.r7562_p2521_p2950                                                                        
1      mc15_5TeV.420042.Sherpa_CT10_SinglePhotonPt70_140.merge.DAOD_HION5.e4878_d1421_r8238_r8052_p3196                                               
2      mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.deriv.NTUP_PILEUP.e6337_e5984_a875_r9364_r9315_p3288                                 
3      mc16_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.deriv.NTUP_PILEUP.e5894_e5984_s3126_r9781_r9778_p3384                                            
4      mc16_13TeV.345890.Sherpa_222_NNPDF30NNLO_eegamma_M_400_1000.deriv.NTUP_PILEUP.e6585_e5984_a875_r10201_r10210_p3384                             
5      mc15_13TeV.395242.MGPy8EG_A14N23LO_SM_N2C1m_160_150_2L2MET75C1M157p5_MadSpin.evgen.log.e6588                                                   
6      data15_13TeV.00281070.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372                                                                        
7      mc16_13TeV.410644.PowhegPythia8EvtGen_A14_singletop_schan_lept_top.deriv.NTUP_PILEUP.e6527_e5984_a875_r10201_r10210_p3384                      
8      mc16_13TeV.364194.Sherpa_221_NNPDF30NNLO_Wtaunu_MAXHTPTV280_500_CFilterBVeto.deriv.DAOD_HIGG4D4.e5340_s3126_r9364_r9315_p3401                  
9      mc15_13TeV.393513.MGPy8EG_A14N23LO_SM_N2N1_155_150_2LMET50_MadSpin.evgen.log.e5512                                                             
10     mc16_13TeV.345747.MadGraphPythia8EvtGen_A14NNPDF23_Zbb_Np1_mVBFfilt.merge.NTUP_PILEUP.e6347_e5984_a875_r9364_r9315_p3288_p3126                 
11     mc16_13TeV.344774.Sherpa_221_NNPDF30NNLO_Ztautau_MAXHTPTV0_70_l15h20.deriv.NTUP_PILEUP.e5585_e5984_s3126_r10201_r10210_p3384                   
12     mc16_13TeV.361022.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ2W.deriv.NTUP_PILEUP.e3668_s3126_r10211_r10210_p3384                                     
13     mc15_13TeV.395088.MGPy8EG_A14N23LO_SM_N2C1m_215_200_2L2MET75_MadSpin.evgen.log.e6587                                                           
14     mc16_13TeV.364203.Sherpa_221_NN30NNLO_Zmm_Mll10_40_MAXHTPTV280_E_CMS_BFilter.merge.NTUP_PILEUP.e5421_e5984_s3126_r10201_r10210_p3384_p3385     
15     mc16_13TeV.301058.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYtautau_5000M.deriv.DAOD_HIGG4D4.e3649_s3126_r9364_r9315_p3401                             
16     mc16_13TeV.301034.PowhegPythia8EvtGen_AZNLOCTEQ6L1_DYmumu_3000M3500.merge.NTUP_PILEUP.e3649_s3126_r9364_r9315_p3127_p3126                      
17     mc16_13TeV.363360.Sherpa_221_NNPDF30NNLO_WplvWmqq.merge.NTUP_PILEUP.e5983_s3126_r9364_r9315_p3127_p3126                                        
18     mc16_13TeV.363152.MGPy8EG_N30NLO_Zee_Ht70_140_BFilter.merge.NTUP_PILEUP.e4866_s3126_r9364_r9315_p3127_p3126                                    
19     mc16_13TeV.363123.MGPy8EG_N30NLO_Zmumu_Ht0_70_CVetoBVeto.merge.NTUP_PILEUP.e4649_e5984_a875_r9364_r9315_p3288_p3126                            
20     mc16_13TeV.363358.Sherpa_221_NNPDF30NNLO_WqqZll.merge.NTUP_PILEUP.e5525_s3126_r9781_r9778_p3384_p3385_tid12840292_00                           
21     mc16_13TeV.363129.MGPy8EG_N30NLO_Zmumu_Ht140_280_CVetoBVeto.deriv.DAOD_HIGG4D1.e4649_s3126_r10201_r10210_p3415                                 
22     mc16_13TeV.364545.Sherpa_222_NNPDF30NNLO_SinglePhoton_pty_280_500.merge.NTUP_PILEUP.e5938_s3126_r9364_r9315_p3127_p3126                        
23     mc15_13TeV.361029.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ9W.merge.DAOD_JETM6.e3569_s2576_s2132_r7772_r7676_p3297                                  
24     mc16_13TeV.341283.PowhegPythia8EvtGen_CT10_AZNLOCTEQ6L1_ggH1200NW_ZZ4lep.deriv.NTUP_PILEUP.e3940_e5984_a875_r10201_r10210_p3384                
25     mc15_13TeV.393683.MGPy8EG_A14N23LO_C1C1_WW_400p0_100p0_2L8.evgen.log.e5639_tid10253945_00                                                      
26     mc16_13TeV.341460.PowhegPy8EG_CT10_AZNLO_ZH125J_MINLO_vt

In [10]:
df_sq = df_sq.withColumn("join_key_sq", concat(df_sq["dataset"], df_sq["datasetScope"]))


In [11]:
df_sq.limit(5).toPandas()

dataset  \
0  data15_13TeV.00284285.physics_Main.merge.DAOD_HIGG4D4.r7562_p2521_p2950                                              
1  mc15_5TeV.420042.Sherpa_CT10_SinglePhotonPt70_140.merge.DAOD_HION5.e4878_d1421_r8238_r8052_p3196                     
2  mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.deriv.NTUP_PILEUP.e6337_e5984_a875_r9364_r9315_p3288       
3  mc16_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.deriv.NTUP_PILEUP.e5894_e5984_s3126_r9781_r9778_p3384                  
4  mc16_13TeV.345890.Sherpa_222_NNPDF30NNLO_eegamma_M_400_1000.deriv.NTUP_PILEUP.e6585_e5984_a875_r10201_r10210_p3384   

   datasetScope  \
0  data15_13TeV   
1  mc15_5TeV      
2  mc16_13TeV     
3  mc16_13TeV     
4  mc16_13TeV     

                                                                                                                    join_key_sq  
0  data15_13TeV.00284285.physics_Main.merge.DAOD_HIGG4D4.r7562_p2521_p2950data15_13TeV                                           
1  mc15_5TeV.420042.Sherpa_CT10_SinglePhotonPt70_140.merge.DAOD_HION5.e4878_d1421_r8238_r8052_p3196mc15_5TeV                     
2  mc16_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.deriv.NTUP_PILEUP.e6337_e5984_a875_r9364_r9315_p3288mc16_13TeV      
3  mc16_13TeV.364250.Sherpa_222_NNPDF30NNLO_llll.deriv.NTUP_PILEUP.e5894_e5984_s3126_r9781_r9778_p3384mc16_13TeV                 
4  mc16_13TeV.345890.Sherpa_222_NNPDF30NNLO_eegamma_M_400_1000.deriv.NTUP_PILEUP.e6585_e5984_a875_r10201_r10210_p3384mc16_13TeV

In [12]:
df_sq.agg(*(countDistinct(col(c)).alias(c) for c in df_sq.columns)).toPandas()

dataset  datasetScope  join_key_sq
0  10578    23            10578

In [13]:
df_lookup = df_lookup.where("DID_TYPE = 'D'")

In [14]:
df_lookup = df_lookup.withColumn("join_key_lookup", concat(df_lookup["NAME"], df_lookup["SCOPE"]))

In [15]:
df_lookup.limit(5).toPandas()

SCOPE  \
0  user.apmorris   
1  user.apmorris   
2  user.apmorris   
3  user.apmorris   
4  user.apmorris   

                                                                                       NAME  \
0  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.180791832    
1  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2_hist.180791833   
2  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081              
3  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081              
4  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081              

     CHILD_SCOPE  \
0  user.apmorris   
1  user.apmorris   
2  user.apmorris   
3  user.apmorris   
4  user.apmorris   

                                                                                              CHILD_NAME  \
0  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.13518847.000001.log.tgz   
1  user.apmorris.13518847._000001.hist-output.root                                                         
2  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.13508095.000001.log.tgz             
3  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.13508095.000002.log.tgz             
4  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.13508095.000003.log.tgz             

  DID_TYPE CHILD_TYPE LENGTH   BYTES   ADLER32   MD5 RULE_EVALUATION  \
0  D        F          None   194713  7a3ce1cd  None  None             
1  D        F          None   8449    862f3b36  None  None             
2  D        F          None   196467  81fb626a  None  None             
3  D        F          None   199926  93184a84  None  None             
4  D        F          None   195386  4bea696c  None  None             

      UPDATED_AT     CREATED_AT  \
0  1521504986000  1521504960000   
1  1521504969000  1521504960000   
2  1521429572000  1521429568000   
3  1521431669000  1521431653000   
4  1521432738000  1521432704000   

                                                                          GUID  \
0  [197, 197, 33, 131, 178, 97, 76, 109, 133, 13, 90, 49, 147, 122, 96, 121]     
1  [100, 32, 22, 125, 61, 83, 73, 65, 139, 125, 231, 224, 59, 129, 110, 118]     
2  [42, 90, 238, 98, 197, 184, 77, 52, 145, 248, 192, 228, 82, 174, 232, 98]     
3  [148, 30, 111, 64, 131, 116, 70, 173, 173, 98, 4, 37, 247, 207, 3, 131]       
4  [188, 104, 171, 46, 179, 124, 76, 67, 162, 177, 144, 76, 167, 246, 236, 97]   

  EVENTS  \
0  None    
1  None    
2  None    
3  None    
4  None    

                                                                                         join_key_lookup  
0  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2.log.180791832user.apmorris   
1  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950.dv21.noEmul_v2_hist.180791833user.apmorris  
2  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081user.apmorris             
3  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081user.apmorris             
4  user.apmorris.data16_Late.00305723.EXOT15.f729_m1646_p2950_dv21.log.180650081user.apmorris

In [16]:
df_lookup.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_lookup.columns if c not in {'GUID'}]).limit(10).toPandas()

SCOPE  NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  CHILD_TYPE     LENGTH  \
0  0      0     0            0           0         0           640870479   

   BYTES  ADLER32        MD5  RULE_EVALUATION  UPDATED_AT  CREATED_AT  \
0  77     37342    747112897  640869857        0           0            

      EVENTS  join_key_lookup  
0  384404134  0

In [17]:
df_lookup.agg(*(countDistinct(col(c)).alias(c) for c in df_lookup.columns)).toPandas()

SCOPE     NAME  CHILD_SCOPE  CHILD_NAME  DID_TYPE  CHILD_TYPE  LENGTH  \
0  2402   6864365  2221         706896095   1         1           1        

       BYTES    ADLER32   MD5  RULE_EVALUATION  UPDATED_AT  CREATED_AT  \
0  237574566  649969967  1230  2                98250726    103929983    

        GUID  EVENTS  join_key_lookup  
0  706848879  482308  6864436

In [18]:
df_lookup.createOrReplaceTempView("lookup")
df_check = spark.sql("SELECT NAME, count(DISTINCT SCOPE)\
                   FROM lookup\
                   GROUP BY NAME\
                   ORDER BY 2 DESC")
df_check.limit(10).toPandas()

NAME  \
0  ttbar_EXOT7_lep_JET_21NP_JET_EffectiveNP_1__1down_131217                     
1  ttbar_EXOT7_lep_JET_21NP_JET_Flavor_Response__1up_131217                     
2  ttbar_EXOT7_lep_JET_21NP_JET_Flavor_Response__1down_131217                   
3  group.phys-gener.MG.364640.MGPy8EvtGen_aQGCFT0_1_ZWjj_llqq.TXT.mc15_v1_i11   
4  group.phys-gener.MG.364638.MGPy8EvtGen_aQGCFM0_5_ZZjj_vvqq.TXT.mc15_v1_i11   
5  ttbar_EXOT7_had_boosted_JET_21NP_JET_Pileup_OffsetMu__1down_131217           
6  ttbar_EXOT7_lep_JET_21NP_JET_Pileup_OffsetNPV__1up_131217                    
7  ttbar_EXOT7_lep_JET_21NP_JET_EffectiveNP_7__1up_131217                       
8  ttbar_EXOT7_lep_MET_SoftTrk_ScaleUp_131217                                   
9  group.phys-gener.MG.364634.MGPy8EvtGen_aQGCFT0_1_ZZjj_llqq.TXT.mc15_v1_i11   

   count(DISTINCT SCOPE)  
0  2                      
1  2                      
2  2                      
3  2                      
4  2                      
5  2                      
6  2                      
7  2                      
8  2                      
9  2

In [19]:
df_check2 = spark.sql("SELECT DISTINCT NAME, SCOPE\
                   FROM lookup\
                   WHERE NAME = 'group.phys-gener.MG.364652.MGPy8EvtGen_aQGCFM0_5_WWjj_lvqq.TXT.mc15_v1_i11'")
df_check2.toPandas()

NAME  \
0  group.phys-gener.MG.364652.MGPy8EvtGen_aQGCFM0_5_WWjj_lvqq.TXT.mc15_v1_i11   
1  group.phys-gener.MG.364652.MGPy8EvtGen_aQGCFM0_5_WWjj_lvqq.TXT.mc15_v1_i11   

              SCOPE  
0  user.lxu          
1  group.phys-gener

In [20]:
df_sq.count()

10578

In [21]:
df_sq.createOrReplaceTempView("df_sq")
df_sq_check = spark.sql("SELECT COUNT(DISTINCT join_key_sq)\
                         FROM df_sq")
df_sq_check.toPandas()

count(DISTINCT join_key_sq)
0  10578

## show difference between NAME and join_key_lookup

In [22]:
#(df_lookup.select("join_key_lookup").subtract(df_lookup.select("Name"))).toPandas()

In [23]:
df_lookup.groupBy("join_key_lookup")

In [24]:
df_joined = df_sq.join(df_lookup, df_sq.join_key_sq == df_lookup.join_key_lookup, "inner")

In [25]:
df_joined.limit(10).toPandas()

dataset  \
0  data16_cos.00306147.express_express.merge.HIST.x459_h158                                               
1  mc15_13TeV.375088.MadGraph_SM_GG_2600.evgen.TXT.e6549_tid13262469_00                                   
2  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00              
3  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00              
4  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
5  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
6  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
7  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
8  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
9  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   

  datasetScope  \
0  data16_cos    
1  mc15_13TeV    
2  mc15_13TeV    
3  mc15_13TeV    
4  mc15_valid    
5  mc15_valid    
6  mc15_valid    
7  mc15_valid    
8  mc15_valid    
9  mc15_valid    

                                                                                                      join_key_sq  \
0  data16_cos.00306147.express_express.merge.HIST.x459_h158data16_cos                                               
1  mc15_13TeV.375088.MadGraph_SM_GG_2600.evgen.TXT.e6549_tid13262469_00mc15_13TeV                                   
2  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00mc15_13TeV              
3  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00mc15_13TeV              
4  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   
5  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   
6  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   
7  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   
8  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   
9  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00mc15_valid   

        SCOPE  \
0  data16_cos   
1  mc15_13TeV   
2  mc15_13TeV   
3  mc15_13TeV   
4  mc15_valid   
5  mc15_valid   
6  mc15_valid   
7  mc15_valid   
8  mc15_valid   
9  mc15_valid   

                                                                                                   NAME  \
0  data16_cos.00306147.express_express.merge.HIST.x459_h158                                               
1  mc15_13TeV.375088.MadGraph_SM_GG_2600.evgen.TXT.e6549_tid13262469_00                                   
2  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00              
3  mc15_13TeV.393676.MGPy8EG_A14N23LO_C1C1_WW_300p0_100p0_2L8.evgen.log.e5639_tid10253917_00              
4  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
5  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
6  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
7  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
8  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   
9  mc15_valid.410006.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e6634_tid13697751_00   

  CHILD_SCOPE  \
0  data16_cos   
1  mc15_13TeV   
2  mc15_13TeV 

In [26]:
df_joined.createOrReplaceTempView("df_joined")
df_joined_check = spark.sql("SELECT COUNT(DISTINCT join_key_sq)\
                         FROM df_joined")
df_joined_check.toPandas()

count(DISTINCT join_key_sq)
0  784

In [27]:
df_lookup.count()

747114130

In [28]:
df_joined.count()

538518

In [29]:
df_joined.agg(*(countDistinct(col(c)).alias(c) for c in df_joined.columns)).toPandas()

dataset  datasetScope  join_key_sq  SCOPE  NAME  CHILD_SCOPE  CHILD_NAME  \
0  784      17            784          17     784   16           538518       

   DID_TYPE  CHILD_TYPE  LENGTH   BYTES  ADLER32  MD5  RULE_EVALUATION  \
0  1         1           0       468700  538478   0    0                 

   UPDATED_AT  CREATED_AT    GUID  EVENTS  join_key_lookup  
0  145160      359972      538518  19504   784

In [30]:
df_joined_check = df_sq.join(df_lookup, df_sq.join_key_sq == df_lookup.join_key_lookup, "left_outer")

In [33]:
df_joined_check.createOrReplaceTempView("joined_check")
df_joined_check_out = spark.sql("SELECT join_key_lookup, COUNT(*)\
                                 FROM joined_check\
                                 GROUP BY join_key_lookup\
                                 ORDER BY 2 DESC")

In [34]:
df_joined_check_out.limit(10).toPandas()

join_key_lookup  \
0  mc15_13TeV.362455.Sherpa_CT10_Wenu_Pt140_280_BFilter_qsf4.evgen.EVNT.e4542_tid06815745_00mc15_13TeV                   
1  mc16_13TeV.425200.Pythia8EvtGen_A14NNPDF23LO_Gammatautau_MassWeight.simul.HITS.e5468_s3126_tid11363708_00mc16_13TeV   
2  data16_13TeV.00304008.physics_Main.recon.DAOD_RPVLL.r9042_r8669_tid10626848_00data16_13TeV                            
3  data15_13TeV.00284285.physics_Main.recon.DAOD_RPVLL.r9042_r8669_tid10626802_00data15_13TeV                            
4  mc15_5TeV.420000.Hijing_PbPb_5p02TeV_MinBias_Flow_JJFV6.simul.HITS.e4162_s2762_tid06911525_00mc15_5TeV                
5  mc15_13TeV.410000.PowhegPythiaEvtGen_P2012_ttbar_hdamp172p5_nonallhad.evgen.EVNT.e3698_tid05192704_00mc15_13TeV       
6  mc15_14TeV.147913.Pythia8_AU2CT10_jetjet_JZ3W.recon.AOD.e2403_s3185_s3186_r9871_tid13291072_00mc15_14TeV              
7  mc15_13TeV.410450.Powheg_ttbar_hdamp258p75_LHE.evgen.EVNT.e6078_tid11772838_00mc15_13TeV                              
8  None                                                                                                                  
9  data17_13TeV.00338377.physics_Main.merge.AOD.f877_m1885data17_13TeV                                                   

   count(1)  
0  199996    
1  29799     
2  29135     
3  28397     
4  21683     
5  19999     
6  19986     
7  11998     
8  9794      
9  7975

In [44]:
df_joined_check_2 = spark.sql("SELECT join_key_sq\
                                 FROM joined_check\
                                 WHERE join_key_lookup == 'None' OR join_key_lookup IS NULL")

In [45]:
df_joined_check_2.limit(10).toPandas()

join_key_sq
0  data15_13TeV.00281411.physics_Main.merge.DAOD_IDTIDE.r9264_p3083_p3083data15_13TeV                                                      
1  data16_13TeV.00303007.physics_Main.deriv.DAOD_HIGG4D4.r9264_p3083_p3372data16_13TeV                                                     
2  data17_13TeV.00336782.physics_Main.merge.DAOD_IDTIDE.r10258_p3399data17_13TeV                                                           
3  mc15_13TeV.361003.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ3.merge.DAOD_EXOT2.e3569_s2576_s2132_r7772_r7676_p2949mc15_13TeV                  
4  mc15_13TeV.363140.MGPy8EG_N30NLO_Zmumu_Ht700_1000_BFilter.evgen.EVNT.e4649mc15_13TeV                                                    
5  mc15_13TeV.395055.MGPy8EG_A14N23LO_SM_N2C1m_120_80_2L2MET75_MadSpin.evgen.log.e6587mc15_13TeV                                           
6  mc15_14TeV.147807.PowhegPythia8_AU2CT10_Zmumu.recon.ESD.e1564_s2630_s2206_r7409mc15_14TeV                                               
7  mc15_14TeV.147808.PowhegPythia8_AU2CT10_Ztautau.recon.AOD.e2491_s3251_s3252_r10252mc15_14TeV                                            
8  mc16_13TeV.301223.Pythia8EvtGen_A14NNPDF23LO_Zprime_NoInt_mumu_E6Chi5000.merge.NTUP_PILEUP.e3990_s3126_r9364_r9315_p3127_p3126mc16_13TeV
9  mc16_13TeV.301575.Pythia8EvtGen_A14NNPDF23LO_DYmumu_3000M3500.merge.NTUP_PILEUP.e3882_e5984_s3126_r9781_r9778_p3384_p3385mc16_13TeV

In [40]:
df_joined_check_3 = spark.sql("SELECT DISTINCT join_key_sq\
                               FROM joined_check\
                               WHERE join_key_lookup == 'None'")

In [41]:
df_joined_check_3.limit(20).toPandas()

Empty DataFrame
Columns: [join_key_sq]
Index: []